In [1]:
from atlassian import Jira
from bidict import bidict

# jira_info = {
#     'url': 'https://tangramcare.atlassian.net/',
#     'username': 'xxx@ownedoutcomes.com',
#     'api_token': 'xxxxxxxxxxxxxxxx',
# }

jira_info = {
    'url': 'https://tangramcare.atlassian.net/',
    'username': 'awaz@ownedoutcomes.com',
    'api_token': '6Pu9J7zSN4wBqwqgSREsAE08',
}

# jira_info = {
#     'url': 'https://awaz.atlassian.net/',
#     'username': 'wazartur@gmail.com',
#     'api_token': 'typYw7sWGOPzOLvtq5Z91BDC',
# }


# jira standard fields config
jira_standard_fields = bidict({
    'Summary': 'summary',
    'Epic': 'customfield_10008',
    'Description': 'description',
    'Project': 'project',
    'Type': 'issuetype',
    'Priority': 'priority',
    'Labels': 'labels',
    'Status': 'status',
    'Creator': 'creator',
    'Assignee': 'assignee',
    'Parent': 'parent',
})

jira = Jira(
            url=jira_info['url'],
            username=jira_info['username'],
            password=jira_info['api_token'],
            cloud=True)

In [ ]:
issues = jira.jql('project=tt', fields=list(o2_pp_fields.inv.keys()), limit=1000)
print(str(issues['maxResults']))
print(str(len(issues['issues'])))

In [ ]:
jira.issue_update('PP-329', {
            'description': 'check',
            'customfield_11701': '2021-03-26',
            'customfield_11702': '2021-03-27'
        })

In [ ]:
from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet["A1"].value = "Design document:"
sheet["C1"].value = "Link"
sheet["C1"].hyperlink = "https://docs.google.com/document/d/1B5RlsVSZdiuKTVAqhrn7xClewqhzrxhliUZd_yIHUBg/edit?usp=sharing"
sheet["C1"].style = "Hyperlink"

sheet.merge_cells('A3:A4')
sheet['A3'].value = 'WBS'

sheet.merge_cells('B3:B4')
sheet['B3'].value = 'Name'

sheet.merge_cells('C3:C4')
sheet['C3'].value = 'Description'

sheet.merge_cells('D3:D4')
sheet['D3'].value = 'Depends on'

sheet.merge_cells('E3:G3')
sheet['E3'].value = 'Estimated development time [h]'
sheet['E4'].value = 'Designer'
sheet['F4'].value = 'Reviewer'
sheet['G4'].value = 'PM'

sheet.merge_cells('H3:H4')
sheet['H3'].value = 'Assigned developer'

sheet.merge_cells('I3:J3')
sheet['I3'].value = 'Estimated testing time [h]'
sheet['I4'].value = 'Tester'
sheet['J4'].value = 'PM'

sheet.merge_cells('K3:K4')
sheet['K3'].value = 'Assigned tester'

sheet.merge_cells('L3:N3')
sheet['L3'].value = 'Actual development time'
sheet['L4'].value = 'Start time'
sheet['M4'].value = 'Finish time'
sheet['N4'].value = 'Duration [h]'

sheet.merge_cells('O3:P3')
sheet['O3'].value = 'Commit statistics'
sheet['O4'].value = 'Insertions'
sheet['P4'].value = 'Deletions'

sheet.merge_cells('Q3:Q4')
sheet['Q3'].value = 'Estimated code documentation time [h]'

sheet.merge_cells('R3:R4')
sheet['R3'].value = 'Jira ID'

sheet.merge_cells('S3:S4')
sheet['S3'].value = 'Commit ID'

sheet.merge_cells('T3:T4')
sheet['T3'].value = 'Comment'


workbook.save(filename="hello_world.xlsx")

In [13]:
import datetime
def datetime_to_date_converter(date):
    from datetime import datetime
    from dateutil import parser
    return parser.parse(date).strftime("%Y-%m-%d")

def get_schema_from_sheet(sheet): # TODO schema validation
    schema = {}
    schema['offset_row'] = 1 # offset of WBS: TODO search for WBS
    schema['offset_col'] = 0 # offset of WBS: TODO search for WBS
    # schema['header_1'] = [i.value for i in sheet['1']] # TODO apply offsets
    # schema['header_2'] = [i.value for i in sheet['2']] # TODO apply offsets
    # schema['map_to_jira'] = { # TODO something with history (start, finish, duration columns)
    #     'Name': 'summary',
    #     'Description': 'description',
    #     'Jira ID': 'key',
    # }


    # schema['columns'] = { # TODO make it dynamic
    #     'WBS': 'A',
    #     'Name': 'B',
    #     'Description': 'C',
    #     'Depends on': 'D',
    #     'Estimate PM': 'G',
    #     'Jira ID': 'R',
    #     'Start time': 'L',
    #     'Finish time': 'M',
    #     'Duration [h]': 'N'
    # }
    # schema['data_offset_row'] = 3
    
    schema['columns'] = { # TODO make it dynamic
        'WBS': 'A',
        'Name': 'B',
        'Description': 'C',
        'Depends on': 'D',
        'Estimate PM': 'H',
        'Jira ID': 'O',
        'Start time': 'J',
        'Finish time': 'K',
        'Duration [h]': 'L'
    }
    schema['data_offset_row'] = 3

    return schema


def get_default_schema_from_template_xlsx(input_template_file = 'template.xlsx', output_schema_file = None):
    from openpyxl import load_workbook
    import json
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)

    if output_schema_file == None:
        return schema
    else:
        json.dump(schema, open(output_schema_file, 'w'))


def get_issues_from_jira_and_create_xlsx_document_from_template(jira, output_file, project, label, input_template_file = 'template.xlsx'):
    from openpyxl import load_workbook
    import json
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)

    issues = jira.jql('project=' + str(project), fields=list(jira_standard_fields.inv.keys()), limit=1000)
    # issues = jira.jql('project=' + str(project) + ' and labels=' + str(label), fields=list(jira_standard_fields.inv.keys()), limit=1000)

    # TODO


def xlsx_to_jira(jira, project, label, input_filename):
    from openpyxl import load_workbook
    
    workbook = load_workbook(filename=input_template_file)
    sheet = workbook['Tasks']

    schema = get_schema_from_sheet(sheet)
    

def wbs_regex_check(string, separator = '.'):
    import re
    return True if re.match('\A([0-9]+\\' + separator + '?)+\Z', string) else False


def wbs_regex_task(string):
    import re
    return True if re.match('\A[0-9]+\Z', string) else False


def check_wbs_level(wbs, separator = '.'):
    return len(wbs.split(separator))


def get_wbs_ancestor(wbs, separator = '.'):
    if len(wbs.split(wbs)) == 1:
        return wbs.split(wbs)[0]
    else:
        return '.'.join(wbs.split('.')[0:-1])


def check_if_successor(wbs, successor):
    wbs_split = wbs.split('.')
    successor_split = successor.split('.')
    if len(wbs_split) >= len(successor_split):
        return False
    for i in range(len(wbs_split)):
        if wbs_split[i] != successor_split[i]:
            return False
    return True


def get_wbs_lowest_level_task_number(wbs, separator = '.'):
    return wbs.split(separator)[-1]


def expand_wbs_range(start, finish, separator = '.'):
    if check_wbs_level(start, separator) != check_wbs_level(finish, separator):
        print('ERROR: WBS range level differs: ', start, ' - ', finish)
        return []
    if get_wbs_ancestor(start, separator) != get_wbs_ancestor(finish, separator):
        print('ERROR: WBS range ancestor differs: ', start, ' - ', finish)
        return []
    lowest_level_task = list(range(int(get_wbs_lowest_level_task_number(start, separator)), 1 + int(get_wbs_lowest_level_task_number(finish, separator))))
    wbs_tasks = []
    for i in lowest_level_task:
        if get_wbs_ancestor(start, separator) == '':
            wbs_tasks.append(str(i))
        else:
            wbs_tasks.append(get_wbs_ancestor(start, separator) + '.' + str(i))
    return wbs_tasks
    

def parse_dependencies(dependency_str, wbs_separator = '.'):
    wbs_tasks = []
    for i in str(dependency_str).split(','):
        if wbs_regex_check(i.strip(), wbs_separator):
            wbs_tasks.append(i)
        else:
            range = i.split('-')
            if len(range) != 2:
                print("ERROR: It is not a range: ", range)
                continue
            if wbs_regex_check(range[0].strip(), wbs_separator) == False or wbs_regex_check(range[1].strip(), wbs_separator) == False:
                print("ERROR: It is not a range: ", range)
                continue
            wbs_tasks = wbs_tasks + expand_wbs_range(range[0].strip(), range[1].strip(), wbs_separator)
    return wbs_tasks


def get_value_for_wbs(schema, wbs, column_name):
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        if wbs == str(sheet[schema['columns']['WBS'] + str(i)].value):
            return str(sheet[schema['columns'][column_name] + str(i)].value)
        i = i + 1 
    return None


def get_jira_key_for_wbs(schema, wbs):
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        if wbs == str(sheet[schema['columns']['WBS'] + str(i)].value):
            return str(sheet[schema['columns']['Jira ID'] + str(i)].value)
        i = i + 1 
    return None


def get_start_finish_from_child(schema, wbs):
    from dateutil import parser
    min = None
    max = None
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        if check_if_successor(wbs, str(sheet[schema['columns']['WBS'] + str(i)].value)):

            if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
                min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)
            if max is None and sheet[schema['columns']['Finish time'] + str(i)].value is not None:
                max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

            if sheet[schema['columns']['Start time'] + str(i)].value is not None: 
                if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
                    min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)
            if sheet[schema['columns']['Finish time'] + str(i)].value is not None: 
                if parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value) > max:
                    max = parser.parse(sheet[schema['columns']['Finish time'] + str(i)].value)

        i = i + 1 
    return min, max


def get_min_from_start(schema):
    from dateutil import parser
    min = None
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        if min is None and sheet[schema['columns']['Start time'] + str(i)].value is not None:
            min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

        if sheet[schema['columns']['Start time'] + str(i)].value is not None: 
            if parser.parse(sheet[schema['columns']['Start time'] + str(i)].value) < min:
                min = parser.parse(sheet[schema['columns']['Start time'] + str(i)].value)

        i = i + 1 
    return min


def create_or_update_issue(jira, schema, index, project, type, label, parent = None):
    if sheet[schema['columns']['Jira ID'] + str(index)].value is None or sheet[schema['columns']['Jira ID'] + str(index)].value == '': # create issue
        issue = jira.issue_create({
            'summary': sheet[schema['columns']['Name'] + str(index)].value,
            'description': sheet[schema['columns']['Description'] + str(index)].value,
            'project': {'key': project},
            'issuetype': {'name': type},
            'parent': {'key': parent},
            'customfield_11701': sheet[schema['columns']['Start time'] + str(index)].value,
            'customfield_11702': sheet[schema['columns']['Finish time'] + str(index)].value,
            # 'customfield_10004': str(float(sheet[schema['columns']['Estimate PM'] + str(index)].value)/4),
            # 'Priority': 'priority',
            # 'Status': 'status',
            # 'Creator': 'creator',
            # 'Assignee': 'assignee',
        })
        sheet[schema['columns']['Jira ID'] + str(index)].value = issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].hyperlink = jira.url + '/browse/' + issue['key']
        sheet[schema['columns']['Jira ID'] + str(index)].font = Font(underline='single', color='0000FF')
        print(issue)
    else:
        task_json = {
            'summary': sheet[schema['columns']['Name'] + str(index)].value,
            'description': sheet[schema['columns']['Description'] + str(index)].value
        }

        min, max = get_start_finish_from_child(schema, sheet[schema['columns']['WBS'] + str(index)].value)
        if min is not None:
            task_json['customfield_11701'] = min.strftime("%Y-%m-%d")
        if max is not None:
            task_json['customfield_11702'] = max.strftime("%Y-%m-%d")

        if sheet[schema['columns']['Start time'] + str(index)].value is not None and min is None:
            task_json['customfield_11701'] = datetime_to_date_converter(sheet[schema['columns']['Start time'] + str(index)].value)
        if sheet[schema['columns']['Finish time'] + str(index)].value is not None and max is None:
            task_json['customfield_11702'] = datetime_to_date_converter(sheet[schema['columns']['Finish time'] + str(index)].value)

        global_min = get_min_from_start(schema)
        if 'customfield_11701' not in task_json:
            task_json['customfield_11701'] = global_min.strftime("%Y-%m-%d")
        if 'customfield_11702' not in task_json:
            pass
            if sheet[schema['columns']['Estimate PM'] + str(index)].value is not None:
                task_json['customfield_11702'] = (global_min + datetime.timedelta(hours = float(sheet[schema['columns']['Estimate PM'] + str(index)].value)*3)).strftime("%Y-%m-%d")
            else:
                task_json['customfield_11702'] = (global_min + datetime.timedelta(days = 1)).strftime("%Y-%m-%d")


        if sheet[schema['columns']['Duration [h]'] + str(index)].value is not None:
            task_json['customfield_10004'] = float(sheet[schema['columns']['Duration [h]'] + str(index)].value)/4
        else:
            if sheet[schema['columns']['Estimate PM'] + str(index)].value is not None:
                task_json['customfield_10004'] = float(sheet[schema['columns']['Estimate PM'] + str(index)].value)/4
                
        jira.issue_update(sheet[schema['columns']['Jira ID'] + str(index)].value, task_json)

    if sheet[schema['columns']['Depends on'] + str(index)].value is not None:
        if sheet[schema['columns']['Depends on'] + str(index)].value != '':
            for dependent_task_wbs in parse_dependencies(sheet[schema['columns']['Depends on'] + str(index)].value):
                fields = {
                    "issuelinks": [
                        {
                            "add": {
                                "type": {
                                    "name": "Gantt End to Start",
                                    "inward": "has to be done after",
                                    "outward": "has to be done before"
                                },
                                "inwardIssue": {
                                    "key": get_jira_key_for_wbs(schema, dependent_task_wbs.strip())
                                }
                            }
                        }
                    ]
                }
                # if get_jira_key_for_wbs(schema, dependent_task_wbs.strip()) is not None or str(get_jira_key_for_wbs(schema, dependent_task_wbs.strip())) != '':
                #     jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields=fields, notify_users=False)
    jira.edit_issue(issue_id_or_key=sheet[schema['columns']['Jira ID'] + str(index)].value, fields={"labels": [{"add": label}]}, notify_users=False)


def create_or_update_issues(jira, sheet, schema, project, label):
    i = schema['data_offset_row']
    while sheet[schema['columns']['WBS'] + str(i)].value is not None:
        wbs = str(sheet[schema['columns']['WBS'] + str(i)].value)
        if not wbs_regex_check(wbs):
            raise Exception('Problem in WBS number structure: ' + str(wbs))

        # if wbs_regex_task(wbs):
        create_or_update_issue(jira=jira, schema=schema, index=i, project=project, type='Task', label=label)
        # else:
        #     import re
        #     parent = get_jira_key_for_wbs(schema=schema, wbs=re.match('\A[0-9]+', wbs)[0])
        #     if parent is None:
        #         raise Exception('Task doesn\'t have a parent: ' + wbs)
        #     create_or_update_issue(jira=jira, schema=schema, index=i, project=project, type='Sub-task', label=label, parent=parent)
        # if i > schema['data_offset_row'] + 5: # for tests purposes
        #     return
        i = i + 1


In [9]:
from openpyxl import load_workbook
from openpyxl.styles import Font

# filename = 'O2 Grouper Pipeline - Task List.xlsx'
# label = 'P2_pipelines'

filename = 'P2 v3 Replatforming Tasks List.xlsx'
label = 'P2_v3'

# filename = 'P2 v3 Authorization_DS manager - list of tasks.xlsx'
# label = 'Authorization'

# filename = 'P2 testing tasks.xlsx'
# label = 'Tests'

workbook = load_workbook(filename=filename, data_only=True)
sheet = workbook['Tasks']

schema = get_schema_from_sheet(sheet)


In [14]:
create_or_update_issues(jira=jira, sheet=sheet, schema=schema, project='PP', label=label)
print("!!! FINISHED !!!")

Updating issue "PP-226" with "{'summary': 'Analytical API', 'description': None, 'customfield_11701': '2021-02-22', 'customfield_11702': '2021-04-14'}"
Updating issue "PP-247" with "{'summary': 'caseSummaryCollection endpoint', 'description': 'This is the basic functionality with filtering but without: benchmarks, pagination or sorting, some response elements are not present in the outputs', 'customfield_11701': '2021-02-22', 'customfield_11702': '2021-04-01'}"
Updating issue "PP-227" with "{'summary': 'Implement Filter and derived classes - ColumnFilter, DistanceFilter', 'description': None, 'customfield_11701': '2021-02-22', 'customfield_11702': '2021-02-23', 'customfield_10004': 2.0}"
Updating issue "PP-228" with "{'summary': 'Implement extract_filters for the CaseSummaryFilterInputType input', 'description': None, 'customfield_11701': '2021-02-23', 'customfield_11702': '2021-03-01', 'customfield_10004': 8.0}"
Updating issue "PP-229" with "{'summary': 'Implement extract_filters for 

In [15]:
workbook.save(filename=filename)
workbook.close()

In [ ]:
map = {
    "A": "wbs",
    "B": "name",
    "C": "description",
    "D": "dependency",
    "E": "estimated_development_time_designer",
    "F": "estimated_development_time_reviewer",
    "G": "estimated_development_time_pm",
    "H": "assigned_developer",
    "I": "estimated_testing_time_tester",
    "J": "estimated_testing_time_pm",
    "K": "assigned_tester",
    "L": "actual_development_time_start",
    "M": "actual_development_time_finish",
    "N": "actual_development_time_duration",
    "O": "commit_statistics_insertions",
    "P": "commit_statistics_deletions",
    "Q": "code_documentation_time",
    "R": "jira_id",
    "S": "commit_id",
    "T": "sprint",
    "U": "comment"
}

tasks_to_json = []

i = schema['data_offset_row']
while sheet[schema['columns']['WBS'] + str(i)].value is not None:
    if len(str(sheet[schema['columns']['Name'] + str(i)].value)) != 0:
        task = {}
        for column in map:
            task[map[column]] = str(sheet[column + str(i)].value)
        tasks_to_json.append(task)
    i = i + 1 

print(tasks_to_json)

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font

workbook = load_workbook(filename='summary.xlsx')
sheet = workbook['Sheet1']

i = 1
while sheet['A' + str(i)].value is not None:

    print(sheet['A' + str(i)].value)
    print(sheet['c'+str(i)].value + sheet['d'+str(i)].value + sheet['e'+str(i)].value)
    issue = jira.issue_create({
            'summary': 'Assign categories to ' + sheet['d'+str(i)].value + ' testcases',
            # 'description': sheet[schema['columns']['Description'] + str(index)].value,
            'project': {'key': 'PP'},
            'issuetype': {'name': 'Sub-task'},
            'parent': {'key': 'PP-162'},
            # 'Priority': 'priority',
            # 'Status': 'status',
            # 'Creator': 'creator',
            # 'Assignee': 'assignee',
        })

    i = i + 1

In [ ]:
print('February 22, 2021, 5:13 PM')

In [ ]:
from dateutil import parser

parser.parse(None)

In [ ]:
import datetime
parser.parse('2021-5-5') + datetime.timedelta(hours = 5.5)
